<a href="https://colab.research.google.com/github/cff100/MecFLu2/blob/main/C%C3%B3digo_Atualizado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cdsapi
!pip install cartopy
!pip install netcdf4
!pip install openpyxl


  Preparing metadata (setup.py) ... done
  Created wheel for multiurl: filename=multiurl-0.3.1-py3-none-any.whl size=21131 sha256=8178f2ec9a8d47da6ab7d9f11013395a8bc2f5a4aa566b9df9cf001020d12885
  Stored in directory: /root/.cache/pip/wheels/9b/d9/5f/36a39fd10d15b5b2d362ad6dc8a1bd28a3b1e14e08357944bf
Successfully built multiurl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.9 MB/s eta 0:00:00


In [2]:
import math
import cdsapi
import xarray as xr
import pandas as pd
from datetime import datetime, timedelta
!pip install --upgrade cdsapi

In [3]:
# Constantes
PA = 101.325  # Pressão atmosférica ao nível do mar em kPa
k = 1.15e-4   # Constante em s²/m²
g = 9.81      # Aceleração gravitacional em m/s²

# Função para calcular a altura a partir da pressão
def calcular_altura(PA, k, P_h, g):
    # P_h: Pressão a uma altura h (em KPa)
    altura = -math.log(P_h / PA) / (k * g)
    return altura

# Função para converter horário de Brasília para UTC
def brasilia_to_utc(hour_brasilia):
    brasilia_time = datetime.strptime(hour_brasilia, '%H:%M')
    utc_time = brasilia_time + timedelta(hours=3)
    return utc_time.strftime('%H:%M')

# Função para converter temperatura de Kelvin para Celsius
def kelvin_to_celsius(kelvin):
    return kelvin - 273.15


# Níveis de pressão em KPa
pressure_levels = [70.0, 75.0, 77.5, 80.0, 82.5, 85.0, 87.5, 90.0, 92.5, 95.0, 97.5, 100.0]

# Coordenadas das plataformas
platforms = {
    'NAMORADO 2 (PNA-2)': (-22.45073, -40.41175),
    'PETROBRAS 26 (P-26)': (-22.4684, -40.02869),
    'PETROBRAS 32 (P-32)': (-22.2051, -40.1431),
    'PETROBRAS 37 (P-37)': (-22.4868, -40.09779),
    'PETROBRAS IX': (-22.57358, -40.82192),
    'PETROBRAS XIX': (-22.3927, -40.05438),
    'PETROBRAS XXXIII': (-22.37, -40.0267),
    'VERMELHO 1 (PVM-1)': (-22.16065, -40.27872),
    'VERMELHO 2 (PVM-2)': (-22.17535, -40.29147),
}


In [4]:
# Hora de Brasília a ser convertida
brasilia_hour = '03:00'
utc_hour = brasilia_to_utc(brasilia_hour)

# Cria um DataFrame vazio para armazenar os resultados combinados de todas as pressões
combined_df = pd.DataFrame()


# Inicializa a API do CDS
c = cdsapi.Client(url='https://cds-beta.climate.copernicus.eu/api', key='89647e12-cf97-495d-a31b-3c325c5fd352')

# Loop sobre cada nível de pressão
for level in pressure_levels:
    # Nome do arquivo NetCDF para o nível de pressão atual com o horário de Brasília
    nc_filename = f'Era5_2023_{int(level*10)}HPA_CAMPOS_{brasilia_hour}.nc'

    # Requisição dos dados
    dataset = 'reanalysis-era5-pressure-levels'
    request = {
    'product_type': ['reanalysis'],
    'variable': ['temperature', 'u_component_of_wind', 'v_component_of_wind'],
    'year': ['2023'],
    'month': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
    'day': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
    'time': [utc_hour],
    'area': [-27, -45, -20, -35],  # Coordenadas da Bacia de Campos
    'pressure_level': f'{int(level*10)}',  # Converte o nível de kPa para hPa
    'data_format': 'netcdf',
    'download_format': 'unarchived'
}

    c.retrieve(dataset, request, nc_filename)

    # Carrega o arquivo NetCDF usando xarray
    ds = xr.open_dataset(nc_filename, engine='netcdf4')

    # Renomeia a coordenada 'valid_time' para 'time'
    ds = ds.rename({'valid_time': 'time'})

    # Calcula a velocidade do vento (wspd)
    wspd = (ds['u']**2 + ds['v']**2)**0.5

    # Calcula a média da velocidade do vento por estação do ano
    wspd_mean_season = wspd.groupby('time.season').mean(dim=['time'])

    # Calcula a média da temperatura por estação do ano
    temp_mean_season = ds['t'].groupby('time.season').mean(dim=['time'])

    # Calcula a altura correspondente ao nível de pressão atual
    altura = calcular_altura(PA, k, level, g)

    # Cria um DataFrame para armazenar os resultados específicos para o nível de pressão atual
    result_df = pd.DataFrame(index=platforms.keys())

    season_names = {
        'DJF': 'Verão',
        'MAM': 'Outono',
        'JJA': 'Inverno',
        'SON': 'Primavera'
    }

    # Loop sobre as estações do ano
    for season in ['DJF', 'MAM', 'JJA', 'SON']:
        for platform, (lat, lon) in platforms.items():

            # Interpola o valor da média da velocidade do vento
            wind_value = wspd_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()

            # Interpola o valor da média da temperatura em Kelvin
            temp_kelvin = temp_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()

            # Converte a temperatura para Celsius
            temp_celsius = kelvin_to_celsius(temp_kelvin)

            # Armazena os valores de vento e temperatura no DataFrame
            result_df.loc[platform, f'Velocidade_Vento_{season_names[season]}'] = wind_value
            result_df.loc[platform, f'Temperatura_K_{season_names[season]}'] = temp_kelvin
            result_df.loc[platform, f'Temperatura_C_{season_names[season]}'] = temp_celsius



    # Adiciona as colunas de pressão e altura
    result_df['Nível_de_Pressão'] = f'{int(level*10)} HPA'
    result_df['Altitude_(m)'] = altura
    result_df['Horário_Brasília'] = brasilia_hour
    result_df['Horário_UTC'] = utc_hour

    # Define o nome do índice para as plataformas
    result_df.index.name = 'Plataforma'

    # Adiciona o DataFrame do nível de pressão atual ao DataFrame combinado
    combined_df = pd.concat([combined_df, result_df])

    # Fecha o arquivo NetCDF
    ds.close()

# Ajusta a ordem das colunas
combined_df = combined_df[['Nível_de_Pressão', 'Altitude_(m)', 'Horário_Brasília', 'Horário_UTC',
                           'Velocidade_Vento_Verão', 'Temperatura_K_Verão', 'Temperatura_C_Verão',
                           'Velocidade_Vento_Outono', 'Temperatura_K_Outono', 'Temperatura_C_Outono',
                           'Velocidade_Vento_Inverno', 'Temperatura_K_Inverno', 'Temperatura_C_Inverno',
                           'Velocidade_Vento_Primavera', 'Temperatura_K_Primavera', 'Temperatura_C_Primavera']]


# Salvar o DataFrame combinado como um arquivo CSV
csv_filename = f'2023_Combined_Results_{brasilia_hour}.csv'
combined_df.to_csv(csv_filename, encoding='utf-8', index=True)

# Salvar o DataFrame combinado como um arquivo Excel
excel_filename = f'2023_Combined_Results_{brasilia_hour}.xlsx'
combined_df.to_excel(excel_filename, index=True)


2024-09-04 12:07:22,749 INFO Request ID is aed735ff-7427-4bfe-9eb6-7fd7a3041458
INFO:cads_api_client.processing:Request ID is aed735ff-7427-4bfe-9eb6-7fd7a3041458
2024-09-04 12:07:22,926 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 12:10:15,061 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


c4506f12b8ba999dcae14c5a7d53cef3.nc:   0%|          | 0.00/3.02M [00:00<?, ?B/s]

2024-09-04 12:10:19,351 INFO Request ID is 37cb3966-5e9d-4bc3-9023-572c0770f108
INFO:cads_api_client.processing:Request ID is 37cb3966-5e9d-4bc3-9023-572c0770f108
2024-09-04 12:10:19,519 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 12:13:11,786 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


f52d89231551cee43d7532b87bad85bd.nc:   0%|          | 0.00/3.05M [00:00<?, ?B/s]

2024-09-04 12:13:15,181 INFO Request ID is c4222a42-3a82-480c-995e-35273b430180
INFO:cads_api_client.processing:Request ID is c4222a42-3a82-480c-995e-35273b430180
2024-09-04 12:13:15,341 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 12:14:30,580 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


...

ab0e23201d4dcf3f93c0993a7ae913e8.nc:   0%|          | 0.00/3.03M [00:00<?, ?B/s]

2024-09-04 12:26:58,757 INFO Request ID is cca96156-1ca1-4750-9499-8f2873d4a086
INFO:cads_api_client.processing:Request ID is cca96156-1ca1-4750-9499-8f2873d4a086
2024-09-04 12:26:58,917 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 12:28:14,133 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


120e0a098aee984964617f56d721bf78.nc:   0%|          | 0.00/3.03M [00:00<?, ?B/s]

2024-09-04 12:28:18,500 INFO Request ID is 18ff5b0a-f1f9-4f32-8799-e9ad91766ca7
INFO:cads_api_client.processing:Request ID is 18ff5b0a-f1f9-4f32-8799-e9ad91766ca7
2024-09-04 12:28:18,662 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 12:29:33,915 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


524314e96ebb7611ec95d3c104ba65d9.nc:   0%|          | 0.00/3.04M [00:00<?, ?B/s]

In [5]:
# Hora de Brasília a ser convertida
brasilia_hour = '09:00'
utc_hour = brasilia_to_utc(brasilia_hour)

# Cria um DataFrame vazio para armazenar os resultados combinados de todas as pressões
combined_df = pd.DataFrame()


# Inicializa a API do CDS
c = cdsapi.Client(url='https://cds-beta.climate.copernicus.eu/api', key='89647e12-cf97-495d-a31b-3c325c5fd352')

# Loop sobre cada nível de pressão
for level in pressure_levels:
    # Nome do arquivo NetCDF para o nível de pressão atual com o horário de Brasília
    nc_filename = f'Era5_2023_{int(level*10)}HPA_CAMPOS_{brasilia_hour}.nc'

    # Requisição dos dados
    dataset = 'reanalysis-era5-pressure-levels'
    request = {
    'product_type': ['reanalysis'],
    'variable': ['temperature', 'u_component_of_wind', 'v_component_of_wind'],
    'year': ['2023'],
    'month': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
    'day': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
    'time': [utc_hour],
    'area': [-27, -45, -20, -35],  # Coordenadas da Bacia de Campos
    'pressure_level': f'{int(level*10)}',  # Converte o nível de kPa para hPa
    'data_format': 'netcdf',
    'download_format': 'unarchived'
}

    c.retrieve(dataset, request, nc_filename)

    # Carrega o arquivo NetCDF usando xarray
    ds = xr.open_dataset(nc_filename, engine='netcdf4')

    # Renomeia a coordenada 'valid_time' para 'time'
    ds = ds.rename({'valid_time': 'time'})

    # Calcula a velocidade do vento (wspd)
    wspd = (ds['u']**2 + ds['v']**2)**0.5

    # Calcula a média da velocidade do vento por estação do ano
    wspd_mean_season = wspd.groupby('time.season').mean(dim=['time'])

    # Calcula a média da temperatura por estação do ano
    temp_mean_season = ds['t'].groupby('time.season').mean(dim=['time'])

    # Calcula a altura correspondente ao nível de pressão atual
    altura = calcular_altura(PA, k, level, g)

    # Cria um DataFrame para armazenar os resultados específicos para o nível de pressão atual
    result_df = pd.DataFrame(index=platforms.keys())

    season_names = {
        'DJF': 'Verão',
        'MAM': 'Outono',
        'JJA': 'Inverno',
        'SON': 'Primavera'
    }

    # Loop sobre as estações do ano
    for season in ['DJF', 'MAM', 'JJA', 'SON']:
        for platform, (lat, lon) in platforms.items():

            # Interpola o valor da média da velocidade do vento
            wind_value = wspd_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()

            # Interpola o valor da média da temperatura em Kelvin
            temp_kelvin = temp_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()

            # Converte a temperatura para Celsius
            temp_celsius = kelvin_to_celsius(temp_kelvin)

            # Armazena os valores de vento e temperatura no DataFrame
            result_df.loc[platform, f'Velocidade_Vento_{season_names[season]}'] = wind_value
            result_df.loc[platform, f'Temperatura_K_{season_names[season]}'] = temp_kelvin
            result_df.loc[platform, f'Temperatura_C_{season_names[season]}'] = temp_celsius



    # Adiciona as colunas de pressão e altura
    result_df['Nível_de_Pressão'] = f'{int(level*10)} HPA'
    result_df['Altitude_(m)'] = altura
    result_df['Horário_Brasília'] = brasilia_hour
    result_df['Horário_UTC'] = utc_hour

    # Define o nome do índice para as plataformas
    result_df.index.name = 'Plataforma'

    # Adiciona o DataFrame do nível de pressão atual ao DataFrame combinado
    combined_df = pd.concat([combined_df, result_df])

    # Fecha o arquivo NetCDF
    ds.close()

# Ajusta a ordem das colunas
combined_df = combined_df[['Nível_de_Pressão', 'Altitude_(m)', 'Horário_Brasília', 'Horário_UTC',
                           'Velocidade_Vento_Verão', 'Temperatura_K_Verão', 'Temperatura_C_Verão',
                           'Velocidade_Vento_Outono', 'Temperatura_K_Outono', 'Temperatura_C_Outono',
                           'Velocidade_Vento_Inverno', 'Temperatura_K_Inverno', 'Temperatura_C_Inverno',
                           'Velocidade_Vento_Primavera', 'Temperatura_K_Primavera', 'Temperatura_C_Primavera']]


# Salvar o DataFrame combinado como um arquivo CSV
csv_filename = f'2023_Combined_Results_{brasilia_hour}.csv'
combined_df.to_csv(csv_filename, encoding='utf-8', index=True)

# Salvar o DataFrame combinado como um arquivo Excel
excel_filename = f'2023_Combined_Results_{brasilia_hour}.xlsx'
combined_df.to_excel(excel_filename, index=True)

2024-09-04 12:29:38,101 INFO Request ID is 2525b548-be13-4c00-841a-61597a629fc0
INFO:cads_api_client.processing:Request ID is 2525b548-be13-4c00-841a-61597a629fc0
2024-09-04 12:29:38,257 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 12:30:53,495 INFO status has been updated to running
INFO:cads_api_client.processing:status has been updated to running
2024-09-04 12:31:32,135 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


cd5ab8f91bd51b5314fdadae5528f62f.nc:   0%|          | 0.00/3.02M [00:00<?, ?B/s]

2024-09-04 12:31:35,346 INFO Request ID is fcdd8dea-991c-4489-8fe1-a29bcf76fe82
INFO:cads_api_client.processing:Request ID is fcdd8dea-991c-4489-8fe1-a29bcf76fe82
2024-09-04 12:31:35,565 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 12:33:29,464 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


8c6e356444940bffbaf4bcc3d67eba9c.nc:   0%|          | 0.00/3.04M [00:00<?, ?B/s]

2024-09-04 12:33:32,631 INFO Request ID is 2926ec1a-fd19-4c4e-bf32-23ebe38c0eff
INFO:cads_api_client.processing:Request ID is 2926ec1a-fd19-4c4e-bf32-23ebe38c0eff
2024-09-04 12:33:32,793 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 12:34:47,998 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


...

4427dfb1d25d152e5f681d8a49734054.nc:   0%|          | 0.00/3.03M [00:00<?, ?B/s]

2024-09-04 12:44:27,263 INFO Request ID is 250773de-9fe0-4624-8eea-b42c3856e682
INFO:cads_api_client.processing:Request ID is 250773de-9fe0-4624-8eea-b42c3856e682
2024-09-04 12:44:27,420 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 12:44:59,592 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


bbd44206b5e0225990fda20d47e74a29.nc:   0%|          | 0.00/3.03M [00:00<?, ?B/s]

2024-09-04 12:45:02,827 INFO Request ID is 92988232-221b-4ea4-beb6-8978e8f75b05
INFO:cads_api_client.processing:Request ID is 92988232-221b-4ea4-beb6-8978e8f75b05
2024-09-04 12:45:02,986 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 12:45:52,381 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


8c22bb400a61d52427493446a8da43e.nc:   0%|          | 0.00/3.04M [00:00<?, ?B/s]

In [6]:
# Hora de Brasília a ser convertida
brasilia_hour = '15:00'
utc_hour = brasilia_to_utc(brasilia_hour)

# Cria um DataFrame vazio para armazenar os resultados combinados de todas as pressões
combined_df = pd.DataFrame()


# Inicializa a API do CDS
c = cdsapi.Client(url='https://cds-beta.climate.copernicus.eu/api', key='89647e12-cf97-495d-a31b-3c325c5fd352')

# Loop sobre cada nível de pressão
for level in pressure_levels:
    # Nome do arquivo NetCDF para o nível de pressão atual com o horário de Brasília
    nc_filename = f'Era5_2023_{int(level*10)}HPA_CAMPOS_{brasilia_hour}.nc'

    # Requisição dos dados
    dataset = 'reanalysis-era5-pressure-levels'
    request = {
    'product_type': ['reanalysis'],
    'variable': ['temperature', 'u_component_of_wind', 'v_component_of_wind'],
    'year': ['2023'],
    'month': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
    'day': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
    'time': [utc_hour],
    'area': [-27, -45, -20, -35],  # Coordenadas da Bacia de Campos
    'pressure_level': f'{int(level*10)}',  # Converte o nível de kPa para hPa
    'data_format': 'netcdf',
    'download_format': 'unarchived'
}

    c.retrieve(dataset, request, nc_filename)

    # Carrega o arquivo NetCDF usando xarray
    ds = xr.open_dataset(nc_filename, engine='netcdf4')

    # Renomeia a coordenada 'valid_time' para 'time'
    ds = ds.rename({'valid_time': 'time'})

    # Calcula a velocidade do vento (wspd)
    wspd = (ds['u']**2 + ds['v']**2)**0.5

    # Calcula a média da velocidade do vento por estação do ano
    wspd_mean_season = wspd.groupby('time.season').mean(dim=['time'])

    # Calcula a média da temperatura por estação do ano
    temp_mean_season = ds['t'].groupby('time.season').mean(dim=['time'])

    # Calcula a altura correspondente ao nível de pressão atual
    altura = calcular_altura(PA, k, level, g)

    # Cria um DataFrame para armazenar os resultados específicos para o nível de pressão atual
    result_df = pd.DataFrame(index=platforms.keys())

    season_names = {
        'DJF': 'Verão',
        'MAM': 'Outono',
        'JJA': 'Inverno',
        'SON': 'Primavera'
    }

    # Loop sobre as estações do ano
    for season in ['DJF', 'MAM', 'JJA', 'SON']:
        for platform, (lat, lon) in platforms.items():

            # Interpola o valor da média da velocidade do vento
            wind_value = wspd_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()

            # Interpola o valor da média da temperatura em Kelvin
            temp_kelvin = temp_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()

            # Converte a temperatura para Celsius
            temp_celsius = kelvin_to_celsius(temp_kelvin)

            # Armazena os valores de vento e temperatura no DataFrame
            result_df.loc[platform, f'Velocidade_Vento_{season_names[season]}'] = wind_value
            result_df.loc[platform, f'Temperatura_K_{season_names[season]}'] = temp_kelvin
            result_df.loc[platform, f'Temperatura_C_{season_names[season]}'] = temp_celsius



    # Adiciona as colunas de pressão e altura
    result_df['Nível_de_Pressão'] = f'{int(level*10)} HPA'
    result_df['Altitude_(m)'] = altura
    result_df['Horário_Brasília'] = brasilia_hour
    result_df['Horário_UTC'] = utc_hour

    # Define o nome do índice para as plataformas
    result_df.index.name = 'Plataforma'

    # Adiciona o DataFrame do nível de pressão atual ao DataFrame combinado
    combined_df = pd.concat([combined_df, result_df])

    # Fecha o arquivo NetCDF
    ds.close()

# Ajusta a ordem das colunas
combined_df = combined_df[['Nível_de_Pressão', 'Altitude_(m)', 'Horário_Brasília', 'Horário_UTC',
                           'Velocidade_Vento_Verão', 'Temperatura_K_Verão', 'Temperatura_C_Verão',
                           'Velocidade_Vento_Outono', 'Temperatura_K_Outono', 'Temperatura_C_Outono',
                           'Velocidade_Vento_Inverno', 'Temperatura_K_Inverno', 'Temperatura_C_Inverno',
                           'Velocidade_Vento_Primavera', 'Temperatura_K_Primavera', 'Temperatura_C_Primavera']]


# Salvar o DataFrame combinado como um arquivo CSV
csv_filename = f'2023_Combined_Results_{brasilia_hour}.csv'
combined_df.to_csv(csv_filename, encoding='utf-8', index=True)

# Salvar o DataFrame combinado como um arquivo Excel
excel_filename = f'2023_Combined_Results_{brasilia_hour}.xlsx'
combined_df.to_excel(excel_filename, index=True)


2024-09-04 12:45:56,506 INFO Request ID is 4e186717-d916-4c19-82a0-2cd57dace1e9
INFO:cads_api_client.processing:Request ID is 4e186717-d916-4c19-82a0-2cd57dace1e9
2024-09-04 12:45:56,668 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 12:46:46,087 INFO status has been updated to running
INFO:cads_api_client.processing:status has been updated to running
2024-09-04 12:47:11,908 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


b12355714885c5979b7edbf597384d19.nc:   0%|          | 0.00/3.01M [00:00<?, ?B/s]

2024-09-04 12:47:15,254 INFO Request ID is 0cbf5a00-ba9c-4adf-a9e6-6720f5ac7c0a
INFO:cads_api_client.processing:Request ID is 0cbf5a00-ba9c-4adf-a9e6-6720f5ac7c0a
2024-09-04 12:47:15,471 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 12:48:30,712 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


78f5be7656a22eba829172add3955723.nc:   0%|          | 0.00/3.03M [00:00<?, ?B/s]

2024-09-04 12:48:34,047 INFO Request ID is 8812b1ae-2ec3-41d9-8b6c-846ee1ea5faf
INFO:cads_api_client.processing:Request ID is 8812b1ae-2ec3-41d9-8b6c-846ee1ea5faf
2024-09-04 12:48:34,214 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 12:49:25,451 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


...

b2ac1755aeee6ba242abf0ce8e44394c.nc:   0%|          | 0.00/3.03M [00:00<?, ?B/s]

2024-09-04 12:56:31,127 INFO Request ID is 369c19ca-59ca-45f0-bcfb-0f8e3da4a032
INFO:cads_api_client.processing:Request ID is 369c19ca-59ca-45f0-bcfb-0f8e3da4a032
2024-09-04 12:56:31,295 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 12:57:20,712 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


f3480c692dec92737444ec103a206e6a.nc:   0%|          | 0.00/3.04M [00:00<?, ?B/s]

2024-09-04 12:57:24,139 INFO Request ID is 745ba751-4e12-432e-8e11-0b5e7b72a0ff
INFO:cads_api_client.processing:Request ID is 745ba751-4e12-432e-8e11-0b5e7b72a0ff
2024-09-04 12:57:24,305 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 12:58:13,713 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


fc6e3de3730c6c1c6b8990b2b20dcb91.nc:   0%|          | 0.00/3.04M [00:00<?, ?B/s]

In [7]:
# Hora de Brasília a ser convertida
brasilia_hour = '21:00'
utc_hour = brasilia_to_utc(brasilia_hour)

# Cria um DataFrame vazio para armazenar os resultados combinados de todas as pressões
combined_df = pd.DataFrame()


# Inicializa a API do CDS
c = cdsapi.Client(url='https://cds-beta.climate.copernicus.eu/api', key='89647e12-cf97-495d-a31b-3c325c5fd352')

# Loop sobre cada nível de pressão
for level in pressure_levels:
    # Nome do arquivo NetCDF para o nível de pressão atual com o horário de Brasília
    nc_filename = f'Era5_2023_{int(level*10)}HPA_CAMPOS_{brasilia_hour}.nc'

    # Requisição dos dados
    dataset = 'reanalysis-era5-pressure-levels'
    request = {
    'product_type': ['reanalysis'],
    'variable': ['temperature', 'u_component_of_wind', 'v_component_of_wind'],
    'year': ['2023'],
    'month': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
    'day': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
    'time': [utc_hour],
    'area': [-27, -45, -20, -35],  # Coordenadas da Bacia de Campos
    'pressure_level': f'{int(level*10)}',  # Converte o nível de kPa para hPa
    'data_format': 'netcdf',
    'download_format': 'unarchived'
}

    c.retrieve(dataset, request, nc_filename)

    # Carrega o arquivo NetCDF usando xarray
    ds = xr.open_dataset(nc_filename, engine='netcdf4')

    # Renomeia a coordenada 'valid_time' para 'time'
    ds = ds.rename({'valid_time': 'time'})

    # Calcula a velocidade do vento (wspd)
    wspd = (ds['u']**2 + ds['v']**2)**0.5

    # Calcula a média da velocidade do vento por estação do ano
    wspd_mean_season = wspd.groupby('time.season').mean(dim=['time'])

    # Calcula a média da temperatura por estação do ano
    temp_mean_season = ds['t'].groupby('time.season').mean(dim=['time'])

    # Calcula a altura correspondente ao nível de pressão atual
    altura = calcular_altura(PA, k, level, g)

    # Cria um DataFrame para armazenar os resultados específicos para o nível de pressão atual
    result_df = pd.DataFrame(index=platforms.keys())

    season_names = {
        'DJF': 'Verão',
        'MAM': 'Outono',
        'JJA': 'Inverno',
        'SON': 'Primavera'
    }

    # Loop sobre as estações do ano
    for season in ['DJF', 'MAM', 'JJA', 'SON']:
        for platform, (lat, lon) in platforms.items():

            # Interpola o valor da média da velocidade do vento
            wind_value = wspd_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()

            # Interpola o valor da média da temperatura em Kelvin
            temp_kelvin = temp_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()

            # Converte a temperatura para Celsius
            temp_celsius = kelvin_to_celsius(temp_kelvin)

            # Armazena os valores de vento e temperatura no DataFrame
            result_df.loc[platform, f'Velocidade_Vento_{season_names[season]}'] = wind_value
            result_df.loc[platform, f'Temperatura_K_{season_names[season]}'] = temp_kelvin
            result_df.loc[platform, f'Temperatura_C_{season_names[season]}'] = temp_celsius



    # Adiciona as colunas de pressão e altura
    result_df['Nível_de_Pressão'] = f'{int(level*10)} HPA'
    result_df['Altitude_(m)'] = altura
    result_df['Horário_Brasília'] = brasilia_hour
    result_df['Horário_UTC'] = utc_hour

    # Define o nome do índice para as plataformas
    result_df.index.name = 'Plataforma'

    # Adiciona o DataFrame do nível de pressão atual ao DataFrame combinado
    combined_df = pd.concat([combined_df, result_df])

    # Fecha o arquivo NetCDF
    ds.close()

# Ajusta a ordem das colunas
combined_df = combined_df[['Nível_de_Pressão', 'Altitude_(m)', 'Horário_Brasília', 'Horário_UTC',
                           'Velocidade_Vento_Verão', 'Temperatura_K_Verão', 'Temperatura_C_Verão',
                           'Velocidade_Vento_Outono', 'Temperatura_K_Outono', 'Temperatura_C_Outono',
                           'Velocidade_Vento_Inverno', 'Temperatura_K_Inverno', 'Temperatura_C_Inverno',
                           'Velocidade_Vento_Primavera', 'Temperatura_K_Primavera', 'Temperatura_C_Primavera']]


# Salvar o DataFrame combinado como um arquivo CSV
csv_filename = f'2023_Combined_Results_{brasilia_hour}.csv'
combined_df.to_csv(csv_filename, encoding='utf-8', index=True)

# Salvar o DataFrame combinado como um arquivo Excel
excel_filename = f'2023_Combined_Results_{brasilia_hour}.xlsx'
combined_df.to_excel(excel_filename, index=True)


2024-09-04 12:58:17,245 INFO Request ID is 72ebb456-c15e-4126-a5f9-b5671f90e019
INFO:cads_api_client.processing:Request ID is 72ebb456-c15e-4126-a5f9-b5671f90e019
2024-09-04 12:58:17,405 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 12:59:06,788 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


61acc64dc8f64455e145365364da8d9c.nc:   0%|          | 0.00/3.02M [00:00<?, ?B/s]

2024-09-04 12:59:10,049 INFO Request ID is 2d219f8e-25de-45f4-8dbf-c7e0fe98e294
INFO:cads_api_client.processing:Request ID is 2d219f8e-25de-45f4-8dbf-c7e0fe98e294
2024-09-04 12:59:10,205 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 12:59:42,383 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


37c3b85867300139c615b68162addff0.nc:   0%|          | 0.00/3.04M [00:00<?, ?B/s]

2024-09-04 12:59:45,531 INFO Request ID is 424919be-35f7-4a30-9830-aa4c7d2e7ed4
INFO:cads_api_client.processing:Request ID is 424919be-35f7-4a30-9830-aa4c7d2e7ed4
2024-09-04 12:59:45,689 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 13:00:35,094 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


...

86189f4358d76dfa2c15c281b3bf418.nc:   0%|          | 0.00/3.03M [00:00<?, ?B/s]

2024-09-04 13:06:32,326 INFO Request ID is eb6b63a8-5df2-46cc-9457-f490467ccf40
INFO:cads_api_client.processing:Request ID is eb6b63a8-5df2-46cc-9457-f490467ccf40
2024-09-04 13:06:32,490 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 13:07:22,393 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


9b4b4754db2759749fc5a3c3569ab9d9.nc:   0%|          | 0.00/3.03M [00:00<?, ?B/s]

2024-09-04 13:07:26,314 INFO Request ID is 45aca087-b3ad-467d-ab2e-d28d8dd27144
INFO:cads_api_client.processing:Request ID is 45aca087-b3ad-467d-ab2e-d28d8dd27144
2024-09-04 13:07:26,474 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 13:08:15,881 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


cd206b52cfb8723b541e51bb1b7fe4d1.nc:   0%|          | 0.00/3.03M [00:00<?, ?B/s]

In [8]:
import pandas as pd
import glob

# Lista de horários na ordem desejada
horarios_brasilia = ['03:00', '09:00', '15:00', '21:00']

# Lista para armazenar os DataFrames na ordem correta
dataframes = []

# Loop sobre cada horário e carregar o DataFrame correspondente
for horario in horarios_brasilia:
    # Buscar o arquivo CSV correspondente ao horário atual
    csv_file = glob.glob(f'2023_Combined_Results_{horario}.csv')

    if csv_file:
        # Carregar o DataFrame e adicionar à lista
        df = pd.read_csv(csv_file[0])
        dataframes.append(df)

# Concatenar os DataFrames na ordem correta
combined_df = pd.concat(dataframes, ignore_index=True)


# Salvar o DataFrame combinado como CSV
combined_df.to_csv('2023_Combined_Results.csv', index=False)

# Salvar o DataFrame combinado como Excel
combined_df.to_excel('2023_Combined_Results.xlsx', index=False)
